In [47]:
## Este primer bloque solo carga los datos que necesitamos para el entrenamiento.
import numpy as np
import os, struct, array
import matplotlib.pyplot as plt
from IPython.display import Image 

def load_mnist(dataset='training', path='.', digits=np.arange(10)):
    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = array.array("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = array.array("B", fimg.read())
    fimg.close()

    ind = [k for k in range(size) if lbl[k] in digits]
    N = len(ind)

    images = np.zeros((N, rows, cols), dtype=np.uint8)
    labels = np.zeros((N, 1), dtype=np.int8)
    for i in range(len(ind)):
        images[i] = np.array(img[ind[i] * rows * cols: (ind[i] + 1) * rows * cols]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

# read the data
images, labels = load_mnist("training", path='data/mnist/')

In [48]:
# Conozcamos a nuestro data set:
X = np.array(images)
indice = np.random.randint(0, X.shape[0])

print "Los datos de entrada constant de " + `X.shape[0]` + " imagenes de " + `X.shape[1]` + " x " + `X.shape[2]` + " pixeles."

plt.imshow(X[indice], cmap=plt.gray())
plt.axis('on')
plt.title(u"Una imagen en particular de nuestro dataset.")
plt.show()


Los datos de entrada constant de 60000 imagenes de 28 x 28 pixeles.


In [49]:
# Parametros
batch_size = 100
np.random.seed(1)
perceptronsHidden = 100
epochs = 2000
learning_rate = 0.02


# Funciones de activacion / utileria
def one_hot(i):
    return np.eye(10)[i]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

def derivSigmoid(x):
    return x * (1 - x)

# Inicializacion de nuestros pesos

Wxh = 2 * np.random.random((784, perceptronsHidden)) - 1
Why = 2 * np.random.random((perceptronsHidden, 10)) - 1
# Wsy = 2 * np.random.random((10, 10))

In [50]:

# create batches for training
for batch in xrange(len(images) / batch_size):
    X = []
    Y = []
    for i in range(batch * batch_size, (batch + 1) * batch_size):
        # input: data de entrenamiento. dim: batch_size * 784
        X.append(images[i].flatten())
        # expected output: batch_size * 10
        Y.append(one_hot(labels[i][0]))

    # layers:
    l0 = np.array(X)
    l1 = sigmoid(np.dot(l0, Wxh))
    l2 = sigmoid(np.dot(l1, Why))

#     lsoft = softmax(np.dot(l2, Wsy))

    normalized_l2 = np.array([softmax(layer2) for layer2 in l2])
    
    # print "Softmax output size: " + `normalized_l2.shape`
    # todo: apply a better error understaing, like cross-entropy
    output_error = Y - normalized_l2

    #Cross Entropy:
    # C = (- np.array(Y) * np.log(normalized_l2)).mean()
    
    # gradient descent
    l2_delta =  output_error * derivSigmoid(normalized_l2)
    hidden_error = l2_delta.dot(Why.T)
    l1_delta = hidden_error * derivSigmoid(l1)

    # weight adjustment
    Why += learning_rate * l1.T.dot(l2_delta)
    Wxh += learning_rate * l0.T.dot(l1_delta)

    print "error: " + str(np.mean(np.abs(output_error)))

# add the testing dataset for validation and calculate error.


error: 0.179819327743
error: 0.178442939623
error: 0.178548803544
error: 0.178592248554
error: 0.177474199103
error: 0.179178914032
error: 0.177635234681
error: 0.179207626661
error: 0.177028952035
error: 0.177097338182
error: 0.175511591774
error: 0.176339032765
error: 0.176650755951
error: 0.177117318306
error: 0.176789001628
error: 0.176635989583
error: 0.175342065848
error: 0.175741318852
error: 0.174919520862
error: 0.174555927706
error: 0.176483533297
error: 0.173246603955
error: 0.175013673839
error: 0.17467541275
error: 0.174338163166
error: 0.174623144311
error: 0.17635496734
error: 0.175363395552
error: 0.173622409918
error: 0.176362631667
error: 0.176392056262
error: 0.174268155536
error: 0.173778299035
error: 0.172443201366
error: 0.173773978008
error: 0.174450512995
error: 0.173811271256
error: 0.1736146087
error: 0.174561729471
error: 0.174603796244
error: 0.173913611584
error: 0.173333459816
error: 0.174808529432
error: 0.17442053433
error: 0.173968586348
error: 0.173009

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: overflow encountered in exp
